In [74]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix

In [75]:
# Users with Similar Behavior to User 1: 2, 9, 11, 12, ...
# data = {
#     "attraction_id": ["A1", "A10", "A11", "A12", "A13", "A14", "A15", "A17", "A19", "A2", "A20", "A3", "A4", "A5", "A6", "A7", "A8", "A9"],
#     1:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     2:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     3:  [0.0, 4.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#     4:  [5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0],
#     5:  [0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#     6:  [0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0],
#     7:  [0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0],
#     8:  [0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 0.0, 2.0, 0.0, 4.0, 2.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0],
#     9:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     10: [0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 2.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0],
#     # 11: [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 12: [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 13: [5.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0],
#     # 14: [0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0, 1.0, 0.0],
#     # 15: [1.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0],
#     # 16: [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 17: [0.0, 1.0, 4.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 18: [4.0, 0.0, 0.0, 5.0, 0.0, 3.0, 2.0, 0.0, 0.0, 3.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0],
#     # 19: [0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0],
#     # 20: [0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 2.0, 0.0, 4.0, 0.0],
# }

# use 2 loop for now -> find all user id -> find each user rating data 
# ** will change after get all rating api
all_usr_id = []
try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/user/getAllUser"
    headers = {
        "Authorization": f"{os.environ['AUTH_BRANCH_API_KEY']}",
        "Content-Type": "application/json"
    }
    res_all_user = requests.get(url=API_ENDPOINT, headers=headers).json()
    all_usr_id = [cur_user['_id'] for cur_user in res_all_user] 

except Exception as e:
    print("get user failed !")
    print(e)

user_rating_data = [] # [[user_id_1, attraction_id_1, rating],...]
for cur_id in all_usr_id:
    try:
        API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/userrating/get/{cur_id}"
        res_user_rating = requests.get(url=API_ENDPOINT).json()
        
        cur_user_id = res_user_rating['userId']
        
        for cur_user_rating in res_user_rating['rating']:
            
            cur_attraction_id = cur_user_rating['attractionId']
            cur_rating_score = cur_user_rating['rating_score']

            user_rating_data.append([cur_user_id, cur_attraction_id, cur_rating_score])


    except Exception as e:
        print("get user_rating failed !")
        print(e)





get user_rating failed !
'userId'


In [76]:
df = pd.DataFrame(user_rating_data, columns=['user_id', 'attraction_id', 'rating_score'])
df
# Rows = Users, Columns = Attractions
pivot_df = df.pivot_table(values = ['rating_score'], 
                       index = ['user_id'],
                       columns = ['attraction_id'],
                       aggfunc='last',
                       fill_value=0)

ratings_matrix = pivot_df
ratings_matrix

,rating_score
attraction_id,675c20369f570448ebeb4a28
user_id,
677e41b2519209b0aaff4d76,4


In [77]:
# defining the recommendations function to get recommendations by using the similar users' preferences
def recommendations(user_id, ratings_matrix):
    ratings_sparse = csr_matrix(ratings_matrix)

    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(ratings_sparse)

    # Find the user's ratings
    user_index = ratings_matrix.index.tolist().index(user_id)
    
    # Get neighbors
    distances, indices = knn.kneighbors(ratings_sparse[user_index], n_neighbors=3+1)
   
    print(indices)
    print("check sim user")
    for i, idx in enumerate(indices[0]):
        print(f"{idx+1} -> (Similarity: {1 - distances[0][i]:.2f})")

    # Get similar users
    similar_users = indices.flatten()[1:] + 1  # Exclude the user itself
    print("check similar_users", similar_users)

    # Aggregate attraction ratings from similar users
    similar_users_ratings = ratings_matrix.iloc[similar_users].mean(axis=0)
    
    # Exclude already rated attractions by the user
    user_ratings = ratings_matrix.iloc[user_index]
    recommendations = similar_users_ratings[user_ratings == 0]
    
    # Sort recommendations by highest score
    recommendations = recommendations.sort_values(ascending=False)
    
    return recommendations.head(30)  # Return top 5 recommendations

In [78]:
res_recommendations_df = recommendations(1,ratings_matrix)
res_recommendations_df

ValueError: 1 is not in list

In [41]:
res_recommendations = res_recommendations_df.index
res_recommendations

Index(['A20', 'A12', 'A1', 'A14', 'A2', 'A19', 'A4', 'A7', 'A9', 'A15', 'A17',
       'A3', 'A6', 'A5'],
      dtype='object', name='attraction_id')

In [42]:
ratings_matrix

attraction_id,A1,A10,A11,A12,A13,A14,A15,A17,A19,A2,A20,A3,A4,A5,A6,A7,A8,A9
1,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
2,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
3,0.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
5,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0
7,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,0.0,2.0,0.0
8,0.0,1.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,4.0,2.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0
9,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
10,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,2.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
